In [ ]:
from datetime import datetime
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import os
import getpass
import json

# -----------------------------
# Step 0: Settings (Placeholders)
# -----------------------------
# Separate folders for pre and post-event downloads
pre_output_dir  = r"PATH_TO_OUTPUT_PRE"
post_output_dir = r"PATH_TO_OUTPUT_POST"

os.makedirs(pre_output_dir, exist_ok=True)
os.makedirs(post_output_dir, exist_ok=True)

# GeoJSON ROI file (placeholder)
roi_file = r"PATH_TO_ROI_GEOJSON"

# Sentinel-2 collection name
DATA_COLLECTION = "SENTINEL-2"

# Pre-event and post-event date ranges (YYYY-MM-DD)
PRE_START  = "YYYY-MM-DD"
PRE_END    = "YYYY-MM-DD"
POST_START = "YYYY-MM-DD"
POST_END   = "YYYY-MM-DD"

# -----------------------------
# Step 1: Load ROI and convert to WKT
# -----------------------------
with open(roi_file) as f:
    roi_geojson = json.load(f)

geom = shape(roi_geojson["features"][0]["geometry"])
roi_wkt = geom.wkt
print("ROI WKT:", roi_wkt)

# -----------------------------
# Step 2: User Credentials (Secure)
# -----------------------------
COPERNICUS_USER = input("Enter your CDSE username (email): ")
COPERNICUS_PASS = getpass.getpass("Enter your CDSE password: ")

# -----------------------------
# Step 3: Get CDSE Keycloak Token
# -----------------------------
def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }

    r = requests.post(
        "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
        data=data
    )
    r.raise_for_status()
    return r.json()["access_token"]

# -----------------------------
# Step 4: Sentinel-2 Download Function
# -----------------------------
def download_s2(start_date, end_date, save_folder):
    start_iso = pd.to_datetime(start_date).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    end_iso   = pd.to_datetime(end_date).strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    query_url = (
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products"
        f"?$filter=Collection/Name eq '{DATA_COLLECTION}' "
        f"and OData.CSC.Intersects(area=geography'SRID=4326;{roi_wkt}') "
        f"and ContentDate/Start ge {start_iso} "
        f"and ContentDate/Start lt {end_iso}"
        f"&$count=True&$top=1000"
    )

    token = get_keycloak(COPERNICUS_USER, COPERNICUS_PASS)
    session = requests.Session()
    session.headers.update({"Authorization": f"Bearer {token}"})

    resp = session.get(query_url)
    resp.raise_for_status()
    data = resp.json()

    if "value" not in data or len(data["value"]) == 0:
        print(f"No Sentinel-2 products found for {start_date} to {end_date}.")
        return

    df = pd.DataFrame.from_dict(data["value"])
    df["geometry"] = df["GeoFootprint"].apply(shape)
    gdf = gpd.GeoDataFrame(df).set_geometry("geometry")
    gdf["identifier"] = gdf["Name"].str.split(".").str[0]

    print(f"Found {len(gdf)} products for {start_date} to {end_date}.")

    for _, row in gdf.iterrows():
        try:
            product_id = row["Id"]
            identifier = row["identifier"]
            name = row["Name"]

            url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
            response = session.get(url, allow_redirects=False)

            while response.status_code in (301, 302, 303, 307):
                url = response.headers["Location"]
                response = session.get(url, allow_redirects=False)

            file_resp = session.get(url, allow_redirects=True)
            save_path = os.path.join(save_folder, f"{identifier}.zip")

            with open(save_path, "wb") as f:
                f.write(file_resp.content)

            print(f"Downloaded: {name}")

        except Exception as e:
            print(f"Download failed for {row['Name']}: {e}")

# -----------------------------
# Step 5: Run Downloads
# -----------------------------
print("\n=== Downloading Pre-event Sentinel-2 ===")
download_s2(PRE_START, PRE_END, pre_output_dir)

print("\n=== Downloading Post-event Sentinel-2 ===")
download_s2(POST_START, POST_END, post_output_dir)

print("\n✔ Sentinel-2 pre and post downloads completed.")


ROI WKT: MULTIPOLYGON (((66.96511369797152 24.845567199224934, 66.98814446135799 24.845649156526775, 66.98715097744719 24.821100518748477, 66.9648879061736 24.82114150550496, 66.96511369797152 24.845567199224934)))
